## Random walk code

In [1]:
import numpy as np
from typing import Callable, Iterator, Self
from dataclasses import dataclass


@dataclass(frozen=True)
class Coord:
    x: int
    y: int

    def adjacent_moves(self) -> Iterator[tuple[Self, float]]:
        """Returns valid adjacent moves with their respective probability"""
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            yield Coord(self.x + dx, self.y + dy), 1 / 4


def stopping_time(non_absorbing: list[Coord], initial: Coord = Coord(0, 0)) -> float:
    """Computes the E[absorption time] for the absorbing markov chain of a 2D bounded RW

    Given a list of non-absorbing states coordinates, computes the expected time to a
    non-absorption state or a coordinate not in the list of non-absorbing states.
    This is done by computing the fundamental matrix and then computing the row-wise sum
    of the initial coordinate. This function is easily extensible into N-dimensions if
    the Coord function is modified

    Time complexity: O(n**2.x) where n is the number of non_absorbing states,
        2.x for the matrix inversion
    Space complexity: O(n**2) where n is the number of non_absorbing states

    Parameters
    ----------
    non_absorbing_coords : list[Coord]
        List of non-absorbing coordinates for the N-d random walk
    initial : Coord, optional
        Initial state, by default Coord(0, 0)

    Returns
    -------
    float
        E[absorption time]
    """
    # Flattens the coordinate mappings for the IQR absorbing state matrix
    coords_mapping = {coords: i for i, coords in enumerate(non_absorbing)}

    # Only the q_matrix is necessary to calculate the E[absorption time]
    q_len = len(non_absorbing)
    q_matrix = np.zeros((q_len, q_len))

    for nac in non_absorbing:
        for adj, prob in nac.adjacent_moves():
            # Assumes non-listed coords are absorbing
            if adj not in coords_mapping: continue
            q_matrix[coords_mapping[nac], coords_mapping[adj]] = prob

    fundamental = np.linalg.inv(np.subtract(np.identity(q_len), q_matrix))

    # Gets row sum of fundamental matrix for initial coord
    # In other words, sums up time in other transient states at initial
    return np.sum(fundamental, axis=1)[coords_mapping[initial]]

## Finding transient states code

In [2]:
def setup(boundary: Callable[[Coord], bool]) -> list[Coord]:
    """DFS flood algo to find all discrete coordinates within the boundary condition

    WHY DFS: I didn't feel like importing a deque

    Parameters
    ----------
    boundary : Callable[[Coord], int]
        Determines whether the given coordinate is within the defined boundary

    Returns
    -------
    list[Coord]
        List of transient coordinates in the grid
    """
    non_absorbing = []

    stack = [Coord(0, 0)]
    visited = set()
    while stack:
        last = stack.pop()
        if last in visited: continue

        if boundary(last):
            non_absorbing.append(last)
            stack.extend(adj for adj, _ in last.adjacent_moves())
        visited.add(last)
    return non_absorbing

## Problem 1

In [3]:
def prob_one_bound(pos: Coord) -> bool:
    """Values are divided by 10 for ease of computation."""
    return -2 < pos.x < 2 and -2 < pos.y < 2


prob_one_non_absorbing = setup(prob_one_bound)
prob_one_time = stopping_time(prob_one_non_absorbing)
print(f"Problem one expected time: {prob_one_time}")

Problem one expected time: 4.5


## Problem 2

Problem 2 can be reduced to a one dimensional walk. The line through $ (0, 10) $ and $ (10, 0) $ can be reduced to the line $ x+y=1 $. For all steps up and right, assuming we start at position $ (0, 0) $, the ant will move closer to the line and for all steps down and left, it will move further away, in terms of manhattan distance.

To solve we can begin by making the solution a bit more generic and say the ant will move closer to the line with $ p \in (1/2, 1) $. With this, it will move farther with probability $ 1-p $. For any markov chain, we can write that the time to position x is $ t_x = 1 + \sum_i P_{ix} \ t_i $. From this, we can simplify our problem to $ t_x = 1 + (p)t_{x-1} + (1-p)t_{x+1} $ and $ t_0 = 1 + (1-p)t_{-1} $.

Another property we can use is the relation between $ t_0 $ and $ t_{-1} $. Starting at $ t_{-1} $ means it has to cover twice the distance to reach $ t_1 $ as $ t_0 $. Therefore, we can rewrite $ t_{-1} = 2 t_0 $. Finally, we can solve the above recurrence relation.

\begin{align*}
t_0 &= 1 + (1-p)t_{-1} \\
t_0 &= 1 + 2 (1-p) t_0 \\
t_0 &= \frac{1}{2p-1} \\

\therefore \\
\displaystyle \lim_{p \to \frac{1}{2}^+} \frac{1}{2p-1} &= \infty
\end{align*}

In [4]:
def prob_three_bound(pos: Coord) -> bool:
    """Values are divided by 10 for ease of computation."""
    x_centi, y_centi = pos.x * 10, pos.y * 10
    return ((x_centi - 2.5) / 30) ** 2 + ((y_centi - 2.5) / 40) ** 2 < 1


prob_three_non_absorbing = setup(prob_three_bound)
prob_three_time = stopping_time(prob_three_non_absorbing)
print(f"Problem three expected time: {prob_three_time}")

Problem three expected time: 13.992053058411821
